In [1]:
import pandas as pd
import numpy as np
import scipy.stats
import math
import warnings
warnings.filterwarnings("ignore")


### This routine handles these steps:
#### 1) read in projection data
#### 2) merge with population data for each group
#### 3) converge age groups to total population
#### 4) get update population shares
#### 5) read into gams , run gams 
#### 6) re-import gams results (housing prices, wages, and updates population shares (from logit)
#### 7) prep gams results for incorporation into logit routine
#### 8) re-run shares in stata
#### 9) map out migration flows for top entry exit


### Steps 1-5
##### (1) read in projection data, (2) merge pop data fro each group, (3) translate age group and skill changes to total population pct change (4) get updated gams shares (5) read into gams, and run new counter factual 1 time

In [ ]:
# tmp0_gr=tmp0.groupby(['msa','statefip'],as_index=False).agg({'id':sum})
# bmk0_gr=bmk0.groupby(['msa'],as_index=False).agg({'id':sum})
# msaid_state_lookup=tmp0_gr[['msa','statefip']].merge(bmk0_gr[['msa']],on='msa').groupby(['msa','statefip'],as_index=False).sum()
# msaid_state_lookup.to_excel('/Users/hannahkamen/Downloads/msaid_state_lookup.xlsx')



In [36]:
#new_bmk=pd.read_stata('/Users/hannahkamen/Downloads/population-migration-master/estimation/1_main_specification/acs5yr0610/dta/projection_data_age2_1_wbmk.dta')
    
    
    

In [100]:
list(tmp0['state'].unique())

['texas',
 'michigan',
 'alabama',
 'montana',
 'new york',
 'georgia',
 'mississippi',
 'north dakota',
 'pennsylvania',
 'north carolina',
 'california',
 'virginia',
 'illinois',
 'nebraska',
 'iowa',
 'maryland',
 'kansas',
 'oregon',
 'nevada',
 'ohio',
 'new mexico',
 'missouri',
 'tennessee',
 'florida',
 'minnesota',
 'delaware',
 'massachusetts',
 'louisiana',
 'wisconsin',
 'indiana',
 'south carolina',
 'colorado',
 'new jersey',
 'arkansas',
 'arizona',
 'washington',
 'idaho',
 'oklahoma',
 'kentucky',
 'south dakota',
 'rhode island',
 'maine',
 'utah',
 'connecticut']

In [38]:
###get state age and education estimates

state_split=pd.read_excel('/Users/hannahkamen/Downloads/state_pop_educ_shares.xlsx')
state_age_shares=pd.read_excel('/Users/hannahkamen/Downloads/state_age_shares.xlsx')
msaid_state_lookup=pd.read_excel('/Users/hannahkamen/Downloads/msaid_state_lookup.xlsx')

state_split=state_split[['skl_2','skl_3','skl_4','skl_5','skl_6','skl_7','unskl_2','unskl_3','unskl_4','unskl_5','unskl_6','unskl_7','state','state_pop']]

master=pd.DataFrame()
for a in [2]:
    for i in [0,1]:       
        try:
            del tmp0
        except:
            print("first loop")
        
        #(1) read in projection data, and benchmark population shares

        tmp0=pd.read_stata('/Users/hannahkamen/Downloads/population-migration-master/estimation/1_main_specification/acs5yr0610/dta/projection_data_age2_%s_wbmk.dta'%i)
        
        #############merge state lookup onto both projections 
        sl=pd.read_excel('/Users/hannahkamen/Downloads/statelookup2.xlsx')
        tmp0=tmp0.merge(sl,on='statefip',how='inner')

        #(2) merge pop share estimates from each group

        ########group benchmark and projected by state, sum shares and chosen to get pct change
        tmp=tmp0.groupby(['state'],as_index=False).agg({'fexthot_28':max,'fextcold':max,'share':sum,'share_b':sum})

        ########merge grouped projections with benchmark to get pct change
        tmp['pct_change']=(tmp['share']-tmp['share_b'])/tmp['share_b']

        ########merge pct changes onto population by age

        tmp=tmp.merge(state_age_shares,on='state',how='inner')
        tmp['age_id']= a
        tmp['educ_id']=i
        ########SET AGE GROUP TWO TO 100 PERCENT OF POPULATION FOR NOW
        #tmp['contribution_to_total_change']=tmp['pct_change']*tmp['%s'%a]
        tmp['contribution_to_total_change']=tmp['pct_change']*1
        
        ######## append all age and skill datasets
        master=master.append(tmp)


#(3) translate age group to total population pct change

master_gr=master.groupby(['state','educ_id'],as_index=False).agg({'contribution_to_total_change':sum})




first loop


In [39]:
#(4) export to gams format
master_gr['educ_id']=master_gr['educ_id'].astype(str).str.replace('0','unskl').str.replace('1','skl')
master_gr=master_gr.merge(sl,on='state',how='inner')
master_gr=master_gr.rename(columns={'abbrev':'','educ_id':'sk','contribution_to_total_change':'skill_shr'})
master_gr=master_gr[['','sk','skill_shr']]
master_gr.to_csv('/Users/hannahkamen/Downloads/le0_shock_0.csv',index=False)

In [55]:
#tmp['fexthot_28'].describe()
#tmp['fextcold'].describe()

#### (6) re-import GAMS results, merge with pop changes

In [132]:
##### import GAMS results
y_rpt=pd.read_csv('/Users/hannahkamen/Downloads/y_rpt.csv')
phou_rpt=pd.read_csv('/Users/hannahkamen/Downloads/phou_rpt.csv')
npl_rpt=pd.read_csv('/Users/hannahkamen/Downloads/npl_rpt.csv')

In [133]:
npl_rpt=npl_rpt[['region','skill','value']]
npl_rpt=npl_rpt.rename(columns={'region':'abbrev','value':'pl'})

In [134]:
phou_rpt=phou_rpt[['pct','region']]
phou_rpt=phou_rpt.rename(columns={'region':'ph','pct':'abbrev'})

In [135]:
#####import state pop shares by skilled and unskilled
state_educ=pd.read_excel('/Users/hannahkamen/Downloads/state_educ_shares.xlsx')
del state_educ['statefip']
state_educ=state_educ.merge(sl,on='state',how='inner')
state_educ=state_educ[['statefip','skl','unskl','state','abbrev','state_pop']]

In [136]:
###merge all fields
r_df=phou_rpt.merge(npl_rpt, on='abbrev').merge(master_gr, left_on=['abbrev','skill'], right_on=['','sk']).merge(state_educ,on='abbrev')



In [137]:
####pivot skill pct changes
pop_changes=r_df.pivot(index='state',columns='sk',values='skill_shr').reset_index()
pop_changes=pop_changes.rename(columns={'skl':'skl_pct_delta','unskl':'unskl_pct_delta'})

#####pivot labor prices
pl_changes=r_df.pivot(index='state',columns='sk',values='pl').reset_index()
pl_changes=pl_changes.rename(columns={'skl':'pl_skl','unskl':'pl_unskl'})

In [138]:
r_df=r_df[['statefip','abbrev','state','skill','pl','ph','skl','unskl','state_pop']]
r_df=r_df.merge(pop_changes,on='state').merge(pl_changes,on='state')
r_df['skilled_level_change']=(r_df['skl']*r_df['state_pop']) + (r_df['skl']*r_df['state_pop']*r_df['skl_pct_delta'])
r_df['unskilled_level_change']=(r_df['unskl']*r_df['state_pop']) + (r_df['unskl']*r_df['state_pop']*r_df['unskl_pct_delta'])
r_df['new_state_pop']=r_df['unskilled_level_change']+r_df['skilled_level_change']



In [141]:
r_df.drop_duplicates?

In [143]:
r_df_lm=r_df.drop_duplicates(subset='state')

In [144]:
r_df_lm.to_stata('/Users/hannahkamen/Downloads/population-migration-master/estimation/1_main_specification/acs5yr0610/dta/gams_dta.dta')




In [146]:
r_df_lm.head()

,statefip,abbrev,state,skill,pl,ph,skl,unskl,state_pop,skl_pct_delta,unskl_pct_delta,pl_skl,pl_unskl,skilled_level_change,unskilled_level_change,new_state_pop
0,19,IA,iowa,skl,1.007405,0.996380,0.390811,0.609189,3046355,-0.096334,-0.111111,1.007405,1.024728,1.075859e+06,1.649605e+06,2.725464e+06
2,12,FL,florida,skl,1.092442,0.991287,0.447951,0.552049,18801310,-0.409469,-0.180052,1.092442,0.968622,4.973491e+06,8.510433e+06,1.348392e+07
4,24,MD,maryland,skl,0.981733,1.001167,0.543044,0.456956,5773552,0.118410,0.045991,0.981733,1.005996,3.506545e+06,2.759592e+06,6.266137e+06
6,39,OH,ohio,skl,0.997464,0.998414,0.405104,0.594896,11536504,-0.023040,-0.044879,0.997464,1.013942,4.565809e+06,6.555014e+06,1.112082e+07
8,47,TN,tennessee,skl,0.999780,0.999079,0.380295,0.619705,6346105,-0.013394,0.002764,0.999780,0.999627,2.381070e+06,3.943582e+06,6.324652e+06


In [ ]:
####translates pop changes by skill into total population changes


In [129]:
np.exp(tmp0['inc_hat'])

0           115799.500000
1            99161.406250
2            24103.830078
3           118211.664062
4           107259.750000
                ...      
16671146     70966.882812
16671147     42222.527344
16671148     27166.675781
16671149     55590.781250
16671150     27816.539062
Name: inc_hat, Length: 16671151, dtype: float32

##### (9) map out migration flows

In [ ]:
#get sum of shares in all other states for people who chose states 1-44
#get ids that live in state s currentlly
master_map=pd.DataFrame()
for s in df_m['state'].unique():
    living_now=[]
    df_lm=df_m[(df_m['state']==s) & (df_m['chosen']==1)]
    ####list of ids currently living in state s
    living_now.append(list(df_lm['id'].unique())[0])
    ###limit dataframe to the ids of people living in state s
    tmp=df_m[df_m['id'].isin(living_now)]
    ####now groupby shares across all states
    tmp2=tmp.groupby(['state'],as_index=False).agg({'share':sum})
    ###tag origin state
    tmp2['living_flag']=s
    master_map=master_map.append(tmp2)
    
######merge with state populaiton
master_map0=master_map.merge(state_split,left_on='living_flag',right_on='state',how='inner')
master_map0=master_map0.rename(columns={'state_x':'moving_to'})

####get estimates for number of people leaving / going to
master_map0['arrive_moving_to']=master_map0['unskl_2']*master_map0['state_pop']

####merge in the msa codes

master_map0_lm=master_map0[master_map0['living_flag']=='texas']

###export texas test

master_map0_lm.to_csv('/Users/hannahkamen/Downloads/texas_out_unskl2.csv')
    
    

In [ ]:
map_dta=pd.read_csv('/Users/hannahkamen/Downloads/census_texas.csv')

In [ ]:
map_dta_lm=map_dta[map_dta['GEOID2'].str.contains(',')]

map_dta_lm['state']=map_dta_lm['GEOID2'].apply(lambda x: x.split(',')[1])
map_dta_lm['state']=map_dta_lm['GEOID2'].apply(lambda x: x.split(',')[1])

In [ ]:
map_dta_lm

In [ ]:
map_dta['state']

In [ ]:
list(map_dta['GEOID2'])

In [ ]:
master_map0.head()

In [ ]:
##test to see what is happening with temperature

In [ ]:
# df_tmp[df_tmp['state']=='washington']

In [ ]:
master_map.sort_values(by='share',ascending=False).head(100)

In [ ]:
master_map.head()

In [ ]:
df_lm

In [ ]:
master_map.head()

In [ ]:
df_test_gre

In [ ]:
df_tmp

In [ ]:
df_test_gre

In [ ]:
# df_m[(df_m['id']==3)& (df_m['state']=='california')][['msa','bpl','chosen','d_s','d_r1','d_r2','statefip','state']]




In [ ]:
df_test_gre['diff']=abs(df_test_gre['share']-df_test_gre['chosen'])

In [ ]:
df_test_gre['diff'].describe()

In [ ]:
# df_tmp=df_m.groupby(['msa'],as_index=False).agg({'fexthot_28':sum,'fextcold':sum,'state':max,'hot':sum,'cold':sum})

In [ ]:
# df_m[['msa','state','hot','cold','fexthot_28','fextcold']].head()

In [ ]:
len(df[(df['id']==4100)]['msa'].unique())

In [ ]:
list()

In [ ]:
logit_ready[logit_ready['chosen']==1]['d_s'].describe()

In [ ]:
df_lm=df[['msa','chosen','share']]

In [ ]:
df[df['chosen']==1]['share'].describe()

#### Import total population and process percent changes for Windc counterfactual

In [ ]:
#orig=pd.read_stata('/Users/hannahkamen/Downloads/population-migration-master/estimation/1_main_specification/acs5yr0610/dta/dta/acs5yr_0610_clr.dta')

#orig.groupby(['ageid'],as_index=False)

In [ ]:
###get shares by college graduates#impo
#educ_shares.to_csv('/Users/hannahkamen/Downloads/le0_shr2.csv',index=False)
# df['pct_col']=np.exp(df['lncoll'])
# df['pct_nc']=1-np.exp(df['lncoll'])
# df['coll_pop']=np.exp(df['lnpop'])*df['pct_col']
# df['population']=np.exp(df['lnpop'])

# df_tot=df.groupby(['statefip'],as_index=False).agg({'population':sum,'coll_pop':sum,'share':sum,'chosen':sum})
# df_tot['state_coll_shr']=df_tot['coll_pop']/df_tot['population']
# df_tot['pct_change']=(df_tot['share']-df_tot['chosen'])/df_tot['chosen']


In [ ]:
df[df['share'].isnull()]

In [ ]:
df

In [ ]:
list(df)

In [ ]:


df_st=df.groupby('statefip',as_index=False).agg({'share':sum,'population':sum,'chosen':sum,'id':'count'})
df_st['chosen'].sum()

In [ ]:
list(df)

In [ ]:
len(df['id'].unique())

In [ ]:
df_st.head()

In [ ]:
len(df_st['id'].unique())

In [ ]:
44*265

In [ ]:
df[df['id']==62973.0]

In [ ]:
len(df)

In [ ]:
df_st.head()

In [ ]:
len(df['statefip'].unique())

In [ ]:
df.groupby('id',as_index=False).agg({'share':sum})['share'].unique()

In [ ]:
df['msa'].unique()

In [ ]:
len(list(df['msa'].unique()))

In [ ]:
df.head()

In [ ]:
data.head()

In [ ]:
st_lkup=data.groupby(['statefip','str_statefip'],as_index=False).sum()[['statefip','str_statefip']]

In [ ]:
list(data)

In [ ]:
data['str_statefip'].unique()

In [ ]:
list(data)

In [ ]:
st_lkup

In [ ]:
data.head()

In [ ]:
len(data)

In [ ]:
list(data)

In [ ]:
len(logit)

In [ ]:
list(logit)

In [ ]:
logit.head()

In [ ]:
q

In [ ]:
logit.head()

In [ ]:
q.head()

In [ ]:
for i in np.arange(2,8,1):
    print(i)

In [ ]:
for i in np.arange(2,8,1):
    df=pd.read_stata('/Users/hannahkamen/Downloads/population-migration-master/estimation/1_main_specification/acs5yr0610/results/temp/2nd_stage_avg_age%s.dta'%i)
    




In [ ]:
df

In [ ]:
secondndstage2.head()

In [ ]:
secondndstage3.head()

In [ ]:
logit['_b_d_r1'].unique()

In [ ]:
logit['d_s'].unique()

In [ ]:
len(cols)

In [ ]:
cols=[x for x in logit.columns if "_b" in x]

In [ ]:
for x in [x for x in logit.columns if "_b" in x]:
    
    print(logit[x].unique())

In [ ]:
logit['d_s']